# Introduction

# Dataset

# Install and Load

In [18]:
%pip install llama-index-llms-openai
%pip install llama-index
%pip install pyvis
%pip install arize-phoenix[evals]
%pip install llama-index-callbacks-arize-phoenix

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
zsh:1: no matches found: arize-phoenix[evals]
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [19]:
import json
from typing import Sequence, List

from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

import nest_asyncio

nest_asyncio.apply()

In [20]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass("Enter your Open API key: ")

Enter your Open API key: ········


In [22]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Document
from llama_index.core import Settings

# Update Settings
Settings.chunk_size = 512
Settings.chunk_overlap = 64
Settings.llm = OpenAI(model="gpt-4o-mini")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")


In [23]:
# Load indices from disk
from llama_index.core import load_index_from_storage

years = [2022, 2021, 2020, 2019]
index_set = {}
for year in years:
    storage_context = StorageContext.from_defaults(
        persist_dir=f"./storage/{year}"
    )
    cur_index = load_index_from_storage(
        storage_context,
    )
    index_set[year] = cur_index

In [24]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

individual_query_engine_tools = []

for year in years:
    # Step 1: Retrieve the query engine for the current year's vector index
    query_engine = index_set[year].as_query_engine()
    print(f"Created query engine for {year}.")

    # Step 2: Define metadata for the query engine tool
    metadata = ToolMetadata(
        name=f"vector_index_{year}",
        description=(
            "Useful for when you want to answer queries about the"
            f" {year} SEC 10-K for Uber."
        ),
    )
    print(f"ToolMetadata for {year}: {metadata}")

    # Step 3: Create a QueryEngineTool for this year
    tool = QueryEngineTool(
        query_engine=query_engine,
        metadata=metadata,
    )
    print(f"Created QueryEngineTool for {year}: {tool}")

    # Step 4: Add the tool to the list
    individual_query_engine_tools.append(tool)
    print(f"Added tool for {year} to the list.\n")



Created query engine for 2022.
ToolMetadata for 2022: ToolMetadata(description='Useful for when you want to answer queries about the 2022 SEC 10-K for Uber.', name='vector_index_2022', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, return_direct=False)
Created QueryEngineTool for 2022: <llama_index.core.tools.query_engine.QueryEngineTool object at 0x28a5e9e90>
Added tool for 2022 to the list.

Created query engine for 2021.
ToolMetadata for 2021: ToolMetadata(description='Useful for when you want to answer queries about the 2021 SEC 10-K for Uber.', name='vector_index_2021', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, return_direct=False)
Created QueryEngineTool for 2021: <llama_index.core.tools.query_engine.QueryEngineTool object at 0x28b07ec50>
Added tool for 2021 to the list.

Created query engine for 2020.
ToolMetadata for 2020: ToolMetadata(description='Useful for when you want to answer queries about the 2020 SEC 10-K for Uber.', n

In [25]:
from llama_index.core.query_engine import SubQuestionQueryEngine

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=individual_query_engine_tools,
)

In [26]:
query_engine_tool = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name="sub_question_query_engine",
        description=(
            "useful for when you want to answer queries that require analyzing"
            " multiple SEC 10-K documents for Uber"
        ),
    ),
)
tools = individual_query_engine_tools + [query_engine_tool]

In [35]:
from llama_index.core.agent import AgentRunner
from llama_index.agent.openai import OpenAIAgentWorker, OpenAIAgent

# Option 1: Initialize OpenAIAgent
agent = OpenAIAgent.from_tools(tools,verbose=True)


# # Option 2: Initialize AgentRunner with OpenAIAgentWorker
# openai_step_engine = OpenAIAgentWorker.from_tools(tools, llm=llm, verbose=True)
# agent = AgentRunner(openai_step_engine)

In [28]:
agent.chat("Hi")

Added user message to memory: Hi


AgentChatResponse(response='Hello! How can I assist you today?', sources=[], source_nodes=[], is_dummy_stream=False, metadata=None)

In [29]:
response = agent.chat(
    "What were some of the biggest risk factors in 2026 for Uber?"
)
print(str(response))

Added user message to memory: What were some of the biggest risk factors in 2026 for Uber?
=== Calling Function ===
Calling function: vector_index_2022 with args: {"input":"risk factors 2026"}
Got output: The provided information does not include specific details about risk factors for the year 2026. It primarily discusses risks related to climate commitments, economic impacts from pandemics, and cybersecurity incidents that have occurred up to 2022. For insights into risk factors specifically for 2026, further information or projections would be necessary.

I couldn't find specific risk factors for Uber in 2026. However, the 2022 SEC 10-K report discusses various risks, including those related to climate commitments, economic impacts from pandemics, and cybersecurity incidents. If you need information on risk factors from a specific year or a broader analysis, please let me know!


# Test ReAct Agent

In [30]:
from llama_index.core.agent import AgentRunner, ReActAgentWorker, ReActAgent

In [34]:
agent_react = ReActAgent.from_tools(tools, verbose=True)


In [32]:
response = agent_react.chat(
    "What were some of the biggest risk factors in 2021 for Uber?"
)
print(str(response))

> Running step 825698c5-66f8-49f0-b9b3-1d2dd7a1f49c. Step input: What were some of the biggest risk factors in 2021 for Uber?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: vector_index_2021
Action Input: {'input': 'What were some of the biggest risk factors for Uber in 2021?'}
Observation: Some of the biggest risk factors for Uber in 2021 included the adverse effects of the COVID-19 pandemic and the actions taken to mitigate it, which impacted various parts of the business. Additionally, the classification of Drivers as employees rather than independent contractors posed a significant risk. The highly competitive nature of the mobility, delivery, and logistics industries, characterized by well-established alternatives and low barriers to entry, also presented challenges. Furthermore, the need to lower fares or service fees to remain competitive, along with the potential inability to attract and maintain a critical ma

In [33]:
response = agent_react.chat(
    "What were the main drivers of Uber's revenue growth or decline in 2021 compared to 2020??"
)
print(str(response))

> Running step 557c2e8c-de9e-4948-aa9f-8b720d1a76d0. Step input: What were the main drivers of Uber's revenue growth or decline in 2021 compared to 2020??
Thought: I need to use a tool to gather information from the 2021 and 2020 SEC 10-K documents for Uber.
Action: sub_question_query_engine
Action Input: {'input': "What were the main drivers of Uber's revenue growth or decline in 2021 compared to 2020?"}
Generated 6 sub questions.
[vector_index_2021] Q: What were the key revenue figures for Uber in 2021 as reported in the SEC 10-K?
[vector_index_2020] Q: What were the key revenue figures for Uber in 2020 as reported in the SEC 10-K?
[vector_index_2021] Q: What factors contributed to revenue growth for Uber in 2021 according to the SEC 10-K?
[vector_index_2021] Q: What factors contributed to revenue decline for Uber in 2021 according to the SEC 10-K?
[vector_index_2021] Q: How did Uber's business segments perform in 2021 compared to 2020?
[vector_index_2021] Q: What were the economic c

In [36]:
response_react = agent_react.chat(
    "What trends can be observed in Uber’s expenses (e.g., marketing, R&D) from 2019 to 2022, and how do these correlate with revenue growth?"
)
print(str(response_react))

> Running step 1a03b2ce-9d42-4bf0-9213-ce48d5b9a599. Step input: What trends can be observed in Uber’s expenses (e.g., marketing, R&D) from 2019 to 2022, and how do these correlate with revenue growth?
Thought: The question requires analyzing multiple SEC 10-K documents for Uber from 2019 to 2022 to identify trends in expenses and their correlation with revenue growth. I will use the sub_question_query_engine tool to gather the necessary information.
Action: sub_question_query_engine
Action Input: {'input': "Analyze Uber's expenses (marketing, R&D, etc.) from 2019 to 2022 and correlate these with revenue growth."}
Generated 12 sub questions.
[vector_index_2019] Q: What were Uber's total marketing expenses in 2019 according to the SEC 10-K?
[vector_index_2020] Q: What were Uber's total marketing expenses in 2020 according to the SEC 10-K?
[vector_index_2021] Q: What were Uber's total marketing expenses in 2021 according to the SEC 10-K?
[vector_index_2022] Q: What were Uber's total mark

In [37]:
response_normal = agent.chat(
    "What trends can be observed in Uber’s expenses (e.g., marketing, R&D) from 2019 to 2022, and how do these correlate with revenue growth?"
)
print(str(response_normal))

Added user message to memory: What trends can be observed in Uber’s expenses (e.g., marketing, R&D) from 2019 to 2022, and how do these correlate with revenue growth?
=== Calling Function ===
Calling function: vector_index_2019 with args: {"input": "Uber's expenses in 2019, including marketing, R&D, and other categories."}
Got output: In 2019, Uber's expenses included various components:

1. **Cost of Revenue**: This primarily consisted of credit card processing fees, bank fees, data center and networking expenses, mobile device and service costs, certain ride insurance costs, payments to Drivers and Restaurants exceeding revenues earned, costs related to Uber Freight transportation services, and amounts related to fare chargebacks and credit card losses.

2. **Operations and Support Expenses**: These expenses mainly comprised compensation costs for employees supporting operations in cities, including general managers and platform user support representatives, as well as costs for cust

# Setup Human In the Loop Chat


In [42]:
def chat_repl(exit_when_done: bool = True):
    """Chat REPL with error handling and debugging."""
    task_message = None
    while task_message != "exit":
        task_message = input(">> Human: ")
        if task_message == "exit":
            break

        try:
            # Step 1: Create a task
            task = agent_react.create_task(task_message)
            print(f"Task created with ID: {task.task_id}")
            print(f"Available tasks: {agent_react.state.task_dict.keys()}")

            response = None
            step_output = None
            message = None

            # Step 2: Run steps for the task
            while message != "exit":
                if task.task_id not in agent_react.state.task_dict:
                    print(f"Task ID {task.task_id} is missing! Skipping...")
                    break

                if message is None or message == "":
                    try:
                        step_output = agent_react.run_step(task.task_id)
                    except KeyError as e:
                        print(f"Error: Task ID not found: {e}")
                        break
                else:
                    step_output = agent_react.run_step(task.task_id, input=message)

                if exit_when_done and step_output.is_last:
                    print(">> Task marked as finished by the agent.")
                    break

                message = input(
                    ">> Add feedback during step? (press enter/leave blank to continue, and type 'exit' to stop): "
                )
                if message == "exit":
                    break

            # Step 3: Finalize and output response
            if step_output is None:
                print(">> No step output. Task discarded.")
            elif not step_output.is_last:
                print(">> Task not completed. Task discarded.")
            else:
                response = agent_react.finalize_response(task.task_id)
                print(f"Agent: {str(response)}")

        except Exception as e:
            print(f"An error occurred: {e}")


In [43]:
chat_repl()

>> Human: How did uber fare compared to competitors in 2021
Task created with ID: 2a20f3fb-9ded-4e7b-afb4-7bd31c9f12db
Available tasks: dict_keys(['def746cb-237f-455e-8500-bc4bdeb781cd', '361f6132-6adc-49a1-81bf-e1930cd030e2', 'cd8b91ec-146c-4b48-a96c-a40794d16669', '2a20f3fb-9ded-4e7b-afb4-7bd31c9f12db'])
> Running step a7b53f9d-a92f-408b-a6ed-84d07339c001. Step input: How did uber fare compared to competitors in 2021
Thought: I need to gather information from Uber's 2021 SEC 10-K to compare its performance with competitors.
Action: vector_index_2021
Action Input: {'input': "Uber's performance metrics and insights in 2021 compared to competitors"}
Observation: In 2021, Uber experienced significant growth in its overall gross bookings, which increased by $32.5 billion, marking a 56% rise compared to the previous year. The delivery segment saw a remarkable 66% growth in gross bookings, driven by heightened demand for food delivery services during the pandemic. Mobility gross bookings al

[vector_index_2022] A: In 2022, Uber's revenue increased by $14.4 billion, or 83%, compared to 2021. This growth was primarily driven by a 28% increase in Gross Bookings, which was 33% on a constant currency basis. The rise in Gross Bookings was largely attributed to higher Mobility Trip volumes as the business recovered from the impacts of COVID-19, along with a significant increase in Freight Gross Bookings due to the acquisition of Transplace in late 2021. Additionally, there was a notable net increase in Mobility revenue resulting from changes in the business model in the UK and adjustments related to historical claims.
[vector_index_2022] A: In 2022, trends in the ride-sharing and delivery sectors highlighted by Uber included a significant recovery and growth in Mobility Adjusted EBITDA, which reached $3.3 billion, reflecting an increase of $1.7 billion compared to 2021. The Delivery segment also showed notable improvement, with Adjusted EBITDA profit of $551 million, a substantia

[vector_index_2020] A: Uber's net income (loss) attributable to Uber Technologies, Inc. for 2020 was a loss of 61% as a percentage of revenue. The basic and diluted net income (loss) per common share for 2020 was reported as $(6.81).
[vector_index_2020] A: In 2020, Uber reported a significant decline in overall gross bookings, which decreased by $7.1 billion, representing an 11% drop compared to 2019. The adjusted EBITDA figures showed a loss of $1,847 million, while the mobility adjusted EBITDA was $1,541 million, and the delivery adjusted EBITDA was $(601) million. The net loss attributable to Uber Technologies, Inc. was $(1,012) million, with a basic net loss per share of $(2.23) and a diluted net loss per share of $(2.26).
[vector_index_2019] A: The provided information does not include specific details about Uber's market share in the ride-sharing and delivery sectors from 2019 to 2022. Therefore, it is not possible to determine how Uber's market share evolved during that period b

>> Add feedback during step? (press enter/leave blank to continue, and type 'exit' to stop): exit
>> Task not completed. Task discarded.
>> Human: 
Task created with ID: 4e2a6895-ed86-476a-817b-086034b6d387
Available tasks: dict_keys(['def746cb-237f-455e-8500-bc4bdeb781cd', '361f6132-6adc-49a1-81bf-e1930cd030e2', 'cd8b91ec-146c-4b48-a96c-a40794d16669', '2a20f3fb-9ded-4e7b-afb4-7bd31c9f12db', '4e2a6895-ed86-476a-817b-086034b6d387'])
> Running step 4727b1ae-09f7-4d61-bca1-0ac02e235d6d. Step input: 
An error occurred: Error code: 400 - {'error': {'message': "Invalid value for 'content': expected a string, got null.", 'type': 'invalid_request_error', 'param': 'messages.[3].content', 'code': None}}
>> Human: exit
